In [20]:
from keras.models import Model
from keras.applications.resnet_v2 import ResNet50V2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from keras.layers import Input, Lambda, Dense, Flatten

Using TensorFlow backend.


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/chest_xray.zip', 'r') as zip_ref:
  zip_ref.extractall('')

In [23]:
import os
os.listdir()


['.config',
 'chest_xray',
 'drive',
 'chest_fp32_model.pb',
 'Convert_tensor_rt.h5',
 'chest_fp6_model.pb',
 'model',
 '__MACOSX',
 'chest_TensorRT_int8.pb',
 'sample_data']

In [0]:
IMAGE_SIZE=[224, 224]
train_data_dir = 'chest_xray/train'
validation_data_dir = 'chest_xray/test'

In [25]:
res = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [0]:
for layer in res.layers:
  layer.trainable = False

In [0]:
folder = glob('chest_xray/train/*')

In [0]:
x = Flatten()(res.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folder), activation='softmax')(x)

In [0]:
model = Model(inputs=res.input, outputs=prediction)

In [30]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [31]:
epochs = 2
batch_size = 40
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [32]:
%%time
model.fit_generator(
    training_set,
    #steps_per_epoch = train_generator.n//batch_size,
    epochs = epochs,
    steps_per_epoch=20,
    validation_data=test_set

)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
20/20 [==============================] - 27s 1s/step - loss: 0.7734 - acc: 0.8900 - val_loss: 3.9542 - val_acc: 0.6987
Epoch 2/2
20/20 [==============================] - 21s 1s/step - loss: 0.5188 - acc: 0.9488 - val_loss: 5.4581 - val_acc: 0.6378
CPU times: user 1min 9s, sys: 1.34 s, total: 1min 11s
Wall time: 47.9 s


In [0]:
model.save('/content/Convert_tensor_rt.h5')

In [0]:
from tensorflow.keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)


In [1]:
from tensorflow.keras.models import load_model
model = load_model('/content/Convert_tensor_rt.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[<tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>]


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K


def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 281 variables.
INFO:tensorflow:Converted 281 variables to const ops.


In [3]:
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)

'model/tf_model.pb'

In [4]:
#converting model to tensorrt
import tensorflow as tf
from tensorflow.python.platform import gfile
sess = tf.Session()
f = gfile.FastGFile("./model/tf_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

Instructions for updating:
Use tf.gfile.GFile.


In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
converter = trt.TrtGraphConverter(
	input_graph_def=graph_def,
	nodes_blacklist=['dense_1/Softmax:0'])
frozen_graph = converter.convert()

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [6]:
tf.train.write_graph(graph_def, "model", "tf_trt_model.pb", as_text=False)

'model/tf_trt_model.pb'

In [7]:
all_nodes=len([1 for n in frozen_graph.node])
print("no. of all_nodes in frozen graph: ",all_nodes)
trt_engine_nodes = len([1 for n in graph_def.node if str(n.op)=='TRTeng'])
print("no. of trt_engine_nodes in tensorrt graph: ",trt_engine_nodes)
all_nodes=len([1 for n in graph_def.node])
print("no. of all_nodes in tensorrt graph: ",all_nodes)

no. of all_nodes in frozen graph:  1548
no. of trt_engine_nodes in tensorrt graph:  0
no. of all_nodes in tensorrt graph:  1628


In [0]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.resnet_v2 import preprocess_input

In [0]:
import os
files = os.listdir('chest_xray/test/PNEUMONIA/')
img1 = image.load_img(r'chest_xray/test/PNEUMONIA/' + files[0], target_size=(224, 224))
img_array1 = image.img_to_array(img1)
img_array_expanded_dims1 = np.expand_dims(img_array1, axis=0)
input_img = preprocess_input(img_array_expanded_dims1)

for i in files[1:70]:
    img2 = image.load_img('chest_xray/test/PNEUMONIA/' + i, target_size=(224, 224))
    img_array2 = image.img_to_array(img2)
    img_array_expanded_dims2 = np.expand_dims(img_array2, axis=0)
    img2 = preprocess_input(img_array_expanded_dims2)

    input_img = np.concatenate((input_img, img2),axis=0)

In [0]:
# function to read a ".pb" model 
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

In [11]:
TENSORRT_MODEL_PATH = "/content/model/tf_trt_model.pb"
import time
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred_ = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT = total_time / n_time_inference
        print("average inference time : ", avg_time_tensorRT)

needed time in inference-0:  0.10517311096191406
needed time in inference-1:  0.10358643531799316
needed time in inference-2:  0.10381317138671875
needed time in inference-3:  0.10467100143432617
needed time in inference-4:  0.10370135307312012
needed time in inference-5:  0.10396289825439453
needed time in inference-6:  0.10717201232910156
needed time in inference-7:  0.10359740257263184
needed time in inference-8:  0.10460042953491211
needed time in inference-9:  0.1043543815612793
needed time in inference-10:  0.10494828224182129
needed time in inference-11:  0.10386157035827637
needed time in inference-12:  0.10411858558654785
needed time in inference-13:  0.10403323173522949
needed time in inference-14:  0.1040339469909668
needed time in inference-15:  0.10443305969238281
needed time in inference-16:  0.10484194755554199
needed time in inference-17:  0.10443592071533203
needed time in inference-18:  0.10449934005737305
needed time in inference-19:  0.10557270050048828
needed time 

In [12]:
# convert (optimize) frozen model to TensorRT model......FP32
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,        # frozen model
    outputs=['dense_1/Softmax:0'],
    max_batch_size=70,           # specify your max batch size
    max_workspace_size_bytes=2*(10**9),    # specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("chest_fp32_model.pb", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
TensorRT model is successfully stored!


In [13]:
import time
# variable
TENSORRT_MODEL_PATH = 'chest_fp32_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred32 = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred_32 = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT_32 = total_time / n_time_inference
        print("average inference time of fp32: ", avg_time_tensorRT_32)

needed time in inference-0:  0.10667896270751953
needed time in inference-1:  0.10476994514465332
needed time in inference-2:  0.1036374568939209
needed time in inference-3:  0.10394096374511719
needed time in inference-4:  0.10422563552856445
needed time in inference-5:  0.10376834869384766
needed time in inference-6:  0.10371851921081543
needed time in inference-7:  0.10458111763000488
needed time in inference-8:  0.10416817665100098
needed time in inference-9:  0.10410642623901367
needed time in inference-10:  0.1038661003112793
needed time in inference-11:  0.10444188117980957
needed time in inference-12:  0.10442829132080078
needed time in inference-13:  0.10410308837890625
needed time in inference-14:  0.1039273738861084
needed time in inference-15:  0.10399174690246582
needed time in inference-16:  0.10386371612548828
needed time in inference-17:  0.10407757759094238
needed time in inference-18:  0.1034536361694336
needed time in inference-19:  0.10373830795288086
needed time in

In [14]:

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,        # frozen model
    outputs=['dense_1/Softmax:0'],
    max_batch_size=70,           # specify your max batch size
    max_workspace_size_bytes=2*(10**9),    # specify the max workspace
    precision_mode="FP16") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("chest_fp6_model.pb", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
TensorRT model is successfully stored!


In [15]:
# variable
TENSORRT_MODEL_PATH = 'chest_fp6_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred16 = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred_16 = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT_16 = total_time / n_time_inference
        print("average inference time of fp16: ", avg_time_tensorRT_16)

needed time in inference-0:  0.10364818572998047
needed time in inference-1:  0.10480713844299316
needed time in inference-2:  0.10406947135925293
needed time in inference-3:  0.10318708419799805
needed time in inference-4:  0.10435891151428223
needed time in inference-5:  0.1040041446685791
needed time in inference-6:  0.10407519340515137
needed time in inference-7:  0.10393071174621582
needed time in inference-8:  0.1047062873840332
needed time in inference-9:  0.10375857353210449
needed time in inference-10:  0.10382652282714844
needed time in inference-11:  0.103363037109375
needed time in inference-12:  0.1042628288269043
needed time in inference-13:  0.10347104072570801
needed time in inference-14:  0.10330963134765625
needed time in inference-15:  0.10398602485656738
needed time in inference-16:  0.10500597953796387
needed time in inference-17:  0.10410189628601074
needed time in inference-18:  0.10398745536804199
needed time in inference-19:  0.10553669929504395
needed time in 

In [16]:
#calibraton
converter = trt.TrtGraphConverter(
    input_graph_def=graph_def,
    nodes_blacklist=['dense_1/Softmax:0'],
    precision_mode='INT8',
    use_calibration=True)
frozen_graph = converter.convert()

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [0]:
frozen_graph = converter.calibrate(
    fetch_names=['dense_1/Softmax:0'],
    num_runs=1,
    feed_dict_fn=lambda: {'input_1:0': input_img})

In [0]:
from tensorflow.python.platform import gfile
with gfile.FastGFile("chest_TensorRT_int8.pb", 'wb') as f:
    f.write(frozen_graph.SerializeToString())

In [19]:
# variable
TENSORRT_MODEL_PATH = 'chest_TensorRT_int8.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred_8 = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred_8 = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT_int8 = total_time / n_time_inference
        print("average inference time of int8: ", avg_time_tensorRT_int8)

needed time in inference-0:  0.1064608097076416
needed time in inference-1:  0.10442042350769043
needed time in inference-2:  0.10556793212890625
needed time in inference-3:  0.10585331916809082
needed time in inference-4:  0.10423660278320312
needed time in inference-5:  0.10444426536560059
needed time in inference-6:  0.1036379337310791
needed time in inference-7:  0.10377073287963867
needed time in inference-8:  0.10321283340454102
needed time in inference-9:  0.10351419448852539
needed time in inference-10:  0.1033315658569336
needed time in inference-11:  0.10394024848937988
needed time in inference-12:  0.10404777526855469
needed time in inference-13:  0.10418415069580078
needed time in inference-14:  0.10410928726196289
needed time in inference-15:  0.10685420036315918
needed time in inference-16:  0.10350227355957031
needed time in inference-17:  0.10400581359863281
needed time in inference-18:  0.10347819328308105
needed time in inference-19:  0.10390806198120117
needed time i